In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import xgboost as xgb
import csv
from sklearn import preprocessing
from keras.layers import LSTM, Dense
from keras.models import Sequential
from keras.layers.core import Masking
from keras.layers.embeddings import Embedding

Using TensorFlow backend.


In [2]:
df1 = pd.read_csv("data/train_preliminary/user.csv")
df2 = pd.read_csv("data/train_preliminary/ad.csv")
df3 = pd.read_csv("data/train_preliminary/click_log.csv")
df = pd.merge(df3, df2, left_on='creative_id', right_on='creative_id')
order = ['user_id', 'time', 'creative_id', 'click_times', 'ad_id', 'product_category', 'advertiser_id']
df = df[order]
df.sort_values(['user_id', 'time'], inplace=True)
df.reset_index(drop=True, inplace=True)
print('read success!')

read success!


In [20]:
x_tmp = np.array(df['creative_id'])
x_tmp = x_tmp.reshape((-1, 1))
x_tmp = np.vstack((x_tmp, np.zeros((29, 1))))
print(x_tmp.shape)
x_tmp = x_tmp.reshape((-1, 100))
model = Sequential()
model.add(Embedding(5000000, 5, input_length=100))
model.compile(loss='MSE', optimizer='adam')
y_tmp = model.predict(x_tmp, verbose=1)
y_tmp = y_tmp.reshape((-1, 5))
x_tmp = pd.DataFrame(y_tmp[:30082788, :], columns=['creative_id_0', 'creative_id_1', 'creative_id_2', 'creative_id_3', 'creative_id_4'])
X = df[['user_id', 'time', 'click_times']]
X = pd.concat([X, x_tmp], axis=1)
x_tmp = np.array(df['ad_id'])
x_tmp = x_tmp.reshape((-1, 1))
x_tmp = np.vstack((x_tmp, np.zeros((29, 1))))
print(x_tmp.shape)
x_tmp = x_tmp.reshape((-1, 100))
model = Sequential()
model.add(Embedding(4000000, 5, input_length=100))
model.compile(loss='MSE', optimizer='adam')
y_tmp = model.predict(x_tmp, verbose=1)
y_tmp = y_tmp.reshape((-1, 5))
x_tmp = pd.DataFrame(y_tmp[:30082788, :], columns=['ad_id_0', 'ad_id_1', 'ad_id_2', 'ad_id_3', 'ad_id_4'])
X = pd.concat([X, x_tmp], axis=1)
x_tmp = np.array(df['product_category'])
x_tmp = x_tmp.reshape((-1, 1))
x_tmp = np.vstack((x_tmp, np.zeros((29, 1))))
print(x_tmp.shape)
x_tmp = x_tmp.reshape((-1, 100))
model = Sequential()
model.add(Embedding(20, 5, input_length=100))
model.compile(loss='MSE', optimizer='adam')
y_tmp = model.predict(x_tmp, verbose=1)
y_tmp = y_tmp.reshape((-1, 5))
x_tmp = pd.DataFrame(y_tmp[:30082788, :], columns=['product_category_0', 'product_category_1',
                                                   'product_category_2', 'product_category_3', 'product_category_4'])
X = pd.concat([X, x_tmp], axis=1)
x_tmp = np.array(df['advertiser_id'])
x_tmp = x_tmp.reshape((-1, 1))
x_tmp = np.vstack((x_tmp, np.zeros((29, 1))))
x_tmp = x_tmp.reshape((-1, 100))
model = Sequential()
model.add(Embedding(70000, 5, input_length=100))
model.compile(loss='MSE', optimizer='adam')
y_tmp = model.predict(x_tmp, verbose=1)
y_tmp = y_tmp.reshape((-1, 5))
x_tmp = pd.DataFrame(y_tmp[:30082788, :], columns=['advertiser_id_0', 'advertiser_id_1',
                                                   'advertiser_id_2', 'advertiser_id_3', 'advertiser_id_4'])
X = pd.concat([X, x_tmp], axis=1)
print('prepare success!')

(30082800, 1)
300828/300828 [==============================] - 4s 12us/step
(30082800, 1)
300828/300828 [==============================] - 4s 13us/step
(30082800, 1)
300828/300828 [==============================] - 3s 10us/step
prepare success!


In [58]:
%%time
X_ready = X.iloc[:1000000, :]
def check_matrix(x):
    x_tmp = x.iloc[:,1:]
    x_tmp = x_tmp.groupby('time').sum()
    x_tmp = x_tmp.reset_index()
    tmp2 = np.zeros((91, 22))
    tmp2 = pd.DataFrame(tmp2)
    tmp2.columns = x_tmp.columns
    for i in range(x_tmp.shape[0]):
        tmp2.iloc[int(x_tmp.iloc[i, 0]) - 1, :] = x_tmp.iloc[i, :]
    return tmp2

X_ready = X_ready.groupby('user_id').apply(check_matrix)
X_ready = X_ready.reset_index(drop=True)
X_ready = X_ready.drop('time', axis = 1)
X_ready = np.array(X_ready).reshape((-1, 91, 21))
X_ready

CPU times: user 6min 43s, sys: 2.14 s, total: 6min 45s
Wall time: 6min 45s


array([[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 1.        , -0.00508491, -0.00340986, ..., -

In [ ]:
X.to_csv('data/train_preliminary/lstm2_5.csv')